## Data Cleaning

The set of codes below will deal with cleaning and pre-processing the gathered data to get it ready to be used for EDA and modeling process. The final products will be saved as CSV files named `tokenize_data.csv` and `clean_data.csv`.
- NOTE: r/xboxone = Xbox | r/PS4 = PS

In [2]:
#Imported Libraries
import re
import string
import numpy as np
import pandas as pd 
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
#Load gathered data 
xbox = pd.read_csv('../datasets/xbox_one.csv', low_memory=False)
ps = pd.read_csv('../datasets/ps_4.csv', low_memory=False)

---

### Checking for Promotional Post

The code block below will check if any of the collected posts are promotional links. According to the API's wiki, promotional posts have their author column labeled as null. 

Reference: https://github.com/reddit-archive/reddit/wiki/JSON

In [4]:
print(f'Number of promotional post in PS subreddit: {ps.author.isnull().sum()}')
print(f'Number of promotional post in Xbox subreddit: {xbox.author.isnull().sum()}')

Number of promotional post in PS subreddit: 0
Number of promotional post in Xbox subreddit: 0


---

### Isolating Desired Columns

For this project, I am only interested in three columns: selftext, title, and subreddit. I isolated these columns and combined the xbox and ps dataframes into one called combine using the code below. 
- **selftext:** the body/content of posts
- **title:** header/title of posts 
- **subreddit:** label that dictates from which subreddit the post is taken from

In [5]:
#Set both dataframes to only contain desired columns 
xbox = xbox[['selftext', 'title', 'subreddit']]
ps = ps[['selftext', 'title', 'subreddit']]

#Combined dataframes together
combine = pd.concat([xbox, ps])
combine = combine.reset_index().drop(columns='index')
combine.head()

,selftext,title,subreddit
0,"Hey, I need help. I just finished coffee talk ...",Game Recommendations,xboxone
1,NaN,Xbox one recognizes nonexistent headset.. pop ...,xboxone
2,NaN,Spyro looking different here,xboxone
3,4 has impossible controls and 6 is the worst g...,Just made the mistake of being buying the resi...,xboxone
4,"I owned a 360, Xbox one og, Xbox one s, and bo...",Diehard fan here. Not a troll post but after a...,xboxone


---

### Treating Null Values (Part 1)
Although most of the selftext column contains missing data, it still holds valuable information. Therefore, I decided to combine it with the title column and created a new column called text. This process is done by imputing all missing values with an empty string and concatenating the two columns. 

Reference: https://github.com/scaress21/reddit_and_quibi/blob/master/code/01A_Gathering_Reddit_Data.ipynb

In [6]:
#Checked for null values 
combine.isnull().sum()

selftext     6483
title           0
subreddit       0
dtype: int64

In [7]:
#Examined the first 5 selftext 
for i in range(0,6):
    self_text = combine.loc[i, 'selftext']
    print(f'Post {i} - {self_text}')

Post 0 - Hey, I need help. I just finished coffee talk (all endings) after binging it all day, and I fell in love. Is there any games on xbox one that are similar to coffee talk?
Post 1 - nan
Post 2 - nan
Post 3 - 4 has impossible controls and 6 is the worst game in the series. I quit as soon as Leon pulled out dual-wield pistols.

Capcom is having a sale on Xbox. I urge anyone without intense nostalgia for the series to stay away. You will regret spending actual money on this junk. I spent 20$ and I’m pissed.
Post 4 - I owned a 360, Xbox one og, Xbox one s, and bought a one x launch. I’m siding with PlayStation the games are top notch literally every exclusive I bought for my og ps4 has run beautifully and I poured every free time I had into beating them they were that good. Play station has a legit I gotta but this console to play this game and enjoy every second of it feeling in it. I wish I could say the same about Xbox the only game that had it was sunset overdrive. They were boug

In [8]:
#Filled null values with an empty string
combine['selftext'].fillna("", inplace=True)

#Add the title and selftext column together
combine['text'] = combine['title'] + " " + combine['selftext']

---

### Setting Target Variable

For this project, my target variable is the subreddit column. Posts coming from the Xbox subreddit are labeled as 0, while those from PS are labeled 1. 

In [9]:
#Binarized the target variable
combine.subreddit = combine.subreddit.map({'xboxone': 0, 'PS4': 1})
combine.head()

,selftext,title,subreddit,text
0,"Hey, I need help. I just finished coffee talk ...",Game Recommendations,0,"Game Recommendations Hey, I need help. I just ..."
1,,Xbox one recognizes nonexistent headset.. pop ...,0,Xbox one recognizes nonexistent headset.. pop ...
2,,Spyro looking different here,0,Spyro looking different here
3,4 has impossible controls and 6 is the worst g...,Just made the mistake of being buying the resi...,0,Just made the mistake of being buying the resi...
4,"I owned a 360, Xbox one og, Xbox one s, and bo...",Diehard fan here. Not a troll post but after a...,0,Diehard fan here. Not a troll post but after a...


---

### Pre-processing

The `cleaning` function below turns text into lowercase, removes enclosed words, numbers, non-English characters, and punctuations. It will also filter out common words such as xbox, microsoft, xbox one, sony, ps3, ps4, ps2,  and ps. The second function, called `word_lemmatize`, will lemmatize words into their root dictionary form. These functions are applied to the text column using a for loop to maximize efficiency.  

Reference 1: https://github.com/adashofdata/nlp-in-python-tutorial/blob/master/1-Data-Cleaning.ipynb 

Reference 2: https://stackoverflow.com/questions/41290028/removing-non-english-words-from-text-using-python

In [10]:
#Function that lowercase text and removes enclosed text, numbers, punctuation, non-English caracters, and common words
def cleaning(word):
    '''Turns words into lowercase, remove text in square brakets, numbers, punctuation, non-English characters, and
    the following words: xbox, xbox one, ps, sony, and microsoft '''
    word = word.lower()
    word = re.sub('\[.*?\]|(ps)\w+|(xbox one)|(xbox)|(microsoft)|(sony)', "", word)
    word = re.sub('[%s]' % re.escape(string.punctuation), "", word)
    word = re.sub('\d+', "", word)
    word = re.sub('[^a-zA-Z]', " ", word)
    return word

#Function that lemmatizes words to root dictionary form
lemmatizer = WordNetLemmatizer()

def word_lemmatize(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

#Applied the functions to the text column 
func_lst = [cleaning, word_lemmatize]
for func in func_lst:
    combine['text'] = combine['text'].map(func)

combine.head()

,selftext,title,subreddit,text
0,"Hey, I need help. I just finished coffee talk ...",Game Recommendations,0,game recommendation hey i need help i just fin...
1,,Xbox one recognizes nonexistent headset.. pop ...,0,recognizes nonexistent headset pop ups in game...
2,,Spyro looking different here,0,spyro looking different here
3,4 has impossible controls and 6 is the worst g...,Just made the mistake of being buying the resi...,0,just made the mistake of being buying the resi...
4,"I owned a 360, Xbox one og, Xbox one s, and bo...",Diehard fan here. Not a troll post but after a...,0,diehard fan here not a troll post but after ac...


---

### Treating Null Values (Part 2)

Due to the removal of non-English characters, there will be rows that only contain empty strings. To fix this problem, I converted those rows into null values and dropped them from the dataset. This method resulted in a final data count of 19973. 

In [11]:
#Converted empty string to null values and dropped them from the dataset
combine['text'].replace("", np.nan, inplace=True)
combine.dropna(subset=['text'],  inplace=True)
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19973 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   selftext   19973 non-null  object
 1   title      19973 non-null  object
 2   subreddit  19973 non-null  int64 
 3   text       19973 non-null  object
dtypes: int64(1), object(3)
memory usage: 780.2+ KB


--- 

### Organizing Data

Since a document-term matrix is needed for the EDA process, I tokenize the text column using sklearn's `CounterVectorizer` and saved it into a new dataframe called `data_df`. During this process, I also removed stop words and added the subreddit column after the vectorization step. 

In [12]:
#Established a document-term matrix
cvec = CountVectorizer(stop_words='english')

cvec_title = cvec.fit_transform(combine['text'])

data_df = pd.DataFrame(cvec_title.toarray(), columns=cvec.get_feature_names())
data_df['subreddit'] = combine['subreddit']
data_df.head()

,aa,aaa,aaaa,aaaaaaaaaaaaaaaaand,aaaah,aaand,aaron,aarongreenberg,aarrrggh,ab,...,zoo,zoom,zoomed,zooming,zoro,zowie,zuccd,zuccing,zucker,zumas
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Saving Data into CSV

In [15]:
data_df.to_csv('../datasets/tokenize_data.csv', index=False)
combine.to_csv('../datasets/cleaned_data.csv', index=False)